In [9]:
import pandas as pd
import numpy as np
from anomalous_edited import *
from function import *

df = pd.read_csv('forestfires.csv')

#Test to see which one is better
#df_normalized = normalization_zscore(df)
##df_normalized = normalization_range(df)

#number of data points
nn = df.shape[0]
#number of features
mm = df.shape[1]

print("Number of data points: ", nn)
print("Number of features: ", mm)

Number of data points:  517
Number of features:  11


In [14]:
# Calculate global statistics
me = [np.mean(df.iloc[:, j]) for j in range(mm)]
mmax = [np.max(df.iloc[:, j]) for j in range(mm)]
mmin = [np.min(df.iloc[:, j]) for j in range(mm)]
ranges = []
normalization = 0  # set to 1 to ignore feature ranges, 0 otherwise
for j in range(mm):
    if normalization:
        ranges.append(1)
    else:
        rng = mmax[j] - mmin[j]
        if rng == 0:
            print("Variable num {} is constant!".format(j))
            rng = 1
        ranges.append(rng)

sy = np.divide((df - me), ranges)
sY = np.array(sy)
d = np.sum(sY * sY)   # total data scatter of normalized data

print("Mean: ", me)
print("Max: ", mmax)
print("Min: ", mmin)

Mean:  [4.669245647969052, 4.299806576402321, 90.64468085106384, 110.87234042553192, 547.9400386847195, 9.021663442940039, 18.88916827852998, 44.28820116054158, 4.017601547388782, 0.021663442940038684, 12.847292069632493]
Max:  [9, 9, 96.2, 291.3, 860.6, 56.1, 33.3, 100, 9.4, 6.4, 1090.84]
Min:  [1, 2, 18.7, 1.1, 7.9, 0.0, 2.2, 15, 0.4, 0.0, 0.0]


In [15]:
# Iterative Anomalous Cluster Algorithm
remains = list(range(nn))  # indices of remaining points
threshold = 25  # minimum cluster size
numberC = 0  # counter of anomalous clusters

# Instead of a single array, store clusters as a list of dictionaries:
ancl = []  # each element: {'cluster': list, 'centroid': list, 'dD': float}

while len(remains) > 0:
    distance = dist(df, remains, ranges, me) # finding normalised distance vector from remains data points to reference 'me'
    ind = np.argmax(distance)
    index = remains[ind]
    centroid = df[index, :]   # initial anomalous center reference point: the one with higher distance
    numberC = numberC + 1
    
    (cluster, centroid) = anomalousPattern(df, remains, ranges, centroid, me) # finding AP cluster
    
    
    censtand = np.divide((np.asarray(centroid) - me), np.asarray(ranges)) # standardised centroid with parameters of the data   
    dD = np.sum(np.divide(censtand * censtand.T * len(cluster) * 100, d)) # cluster contribution (per cent) - (lecture on K-means and iK-means)

    remains = np.setdiff1d(remains, cluster) 
    # update the data structure that keeps everything together
    ancl.append(cluster)   # set of data points in the cluster
    ancl.append(censtand)  # standardised centroid
    ancl.append(dD) # proportion of the data scatter

InvalidIndexError: (slice(None, None, None), 0)

In [ ]:
# Filter clusters by threshold size
filtered_ancl = [ac for ac in ancl if len(ac['cluster']) >= threshold]

if len(filtered_ancl) == 0:
    print('Too great a threshold!!!')
else:
    # For demonstration, print out the clusters and their standardized centroids
    for i, ac in enumerate(filtered_ancl):
        print(f"Cluster {i + 1}:")
        print("  Size:", len(ac['cluster']))
        print("  Centroid (standardized):", np.round(ac['centroid'], 3))
        print("  Cluster contribution (%):", np.round(ac['dD'], 3))